# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_consb').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [0]:
df = sqlContext.sql("SELECT * FROM cruise_ship_info_csv")
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [0]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [0]:
SI = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_Index",
    stringOrderType="frequencyDesc")

In [0]:
idx = SI.fit(df).transform(df)
idx.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_line_Index: double (nullable = false)

In [0]:
idx.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
only showing top 20 rows

In [0]:
ass = VectorAssembler(inputCols=["Age","Tonnage","passengers","length","cabins","passenger_density","Cruise_line_Index"],
                     outputCol='features')



In [0]:
op = ass.transform(idx)

In [0]:
op.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_line_Index: double (nullable = false)
-- features: vector (nullable = true)

In [0]:
dff = op.select('features','crew')
dff.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
train, test = dff.randomSplit([.7,.3])

In [0]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train)
res=lr_model.evaluate(test)
res.residuals.show()

+--------------------+
 residuals|
+--------------------+
-0.21418094349098382|
 0.20842840792719208|
 0.6956940958530415|
-0.39644406022768663|
 -0.7062243424088503|
-0.34375401973083797|
 0.9440715208760331|
 -0.6345298804544033|
-0.47227124414312804|
 -0.6468876910257695|
-0.17592516738785413|
-0.40163463585624104|
-0.19491174872467987|
 0.8968620648817378|
 0.38543385391795126|
 -0.3006797203427949|
 0.8647344791468896|
 0.7873219694914173|
-0.21548111823235905|
-0.00746512117163...|
+--------------------+
only showing top 20 rows

In [0]:
res.rootMeanSquaredError

Out[31]: 0.5833942624296301

In [0]:
res.r2

Out[32]: 0.9763526238089534

In [0]:
tst = test.select('features')
tst.show()

+--------------------+
 features|
+--------------------+
[4.0,220.0,54.0,1...|
[5.0,115.0,35.74,...|
[5.0,122.0,28.5,1...|
[6.0,158.0,43.7,1...|
[7.0,116.0,31.0,9...|
[7.0,158.0,43.7,1...|
[8.0,91.0,22.44,9...|
[9.0,85.0,19.68,9...|
[9.0,90.09,25.01,...|
[9.0,105.0,27.2,8...|
[9.0,116.0,26.0,9...|
[10.0,68.0,10.8,7...|
[10.0,90.09,25.01...|
[11.0,108.977,26....|
[12.0,2.329,0.94,...|
[12.0,50.0,7.0,7....|
[12.0,88.5,21.24,...|
[12.0,91.0,20.32,...|
[13.0,25.0,3.82,5...|
[13.0,30.27699999...|
+--------------------+
only showing top 20 rows

In [0]:
preds = lr_model.transform(tst)
preds.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
[4.0,220.0,54.0,1...| 21.214180943490984|
[5.0,115.0,35.74,...| 11.991571592072807|
[5.0,122.0,28.5,1...| 6.004305904146959|
[6.0,158.0,43.7,1...| 13.996444060227686|
[7.0,116.0,31.0,9...| 12.70622434240885|
[7.0,158.0,43.7,1...| 13.943754019730838|
[8.0,91.0,22.44,9...| 10.055928479123967|
[9.0,85.0,19.68,9...| 9.324529880454403|
[9.0,90.09,25.01,...| 9.162271244143128|
[9.0,105.0,27.2,8...| 11.32688769102577|
[9.0,116.0,26.0,9...| 11.175925167387854|
[10.0,68.0,10.8,7...| 6.761634635856241|
[10.0,90.09,25.01...| 8.77491174872468|
[11.0,108.977,26....| 11.103137935118262|
[12.0,2.329,0.94,...|0.21456614608204871|
[12.0,50.0,7.0,7....| 4.750679720342795|
[12.0,88.5,21.24,...| 9.42526552085311|
[12.0,91.0,20.32,...| 9.202678030508583|
[13.0,25.0,3.82,5...| 3.1654811182323592|
[13.0,30.27699999...| 4.00746512117164|
+--------------------+-------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import corr

In [0]:
df.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+